# Curso de aprendizaje automatizado
PCIC, UNAM

### Tarea 1: Clasificador bayesiano ingenuo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import plotly.express as px

In [2]:
# Descargar el conjunto de datos
url = "https://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/aves.csv"
data = pd.read_csv(url, sep=',', index_col=0, dtype={'y': 'int', 'X1': 'float64', 'X2': 'float64'})

In [3]:
data = data.dropna()

In [4]:
data.head()

,x1,x2,y
0,7.606876,11.421913,0
1,8.300079,12.352260,0
2,6.833771,10.770320,0
3,7.349341,10.903545,0
4,7.442586,11.730364,0


In [5]:
target = data['y'].value_counts()
fig = px.pie(values=target, title='Distribución de clases', names={0: 'Species 1', 1: 'Species 2'})
fig.show()

In [6]:
# Separar características y etiquetas
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Dividir el conjunto de datos en entrenamiento (60%) y temporal (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)

# Dividir el conjunto temporal en validación (50% de 40% = 20%) y prueba (50% de 40% = 20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [25]:
from sklearn.naive_bayes import MultinomialNB

# Entrenar clasificadores bayesianos ingenuos
gnb = MultinomialNB()

gnb.fit(X_train, y_train)

# Predecir y reportar el porcentaje de predicciones correctas en entrenamiento y validación
y_train_pred_gnb = gnb.predict(X_train)
y_val_pred_gnb = gnb.predict(X_val)

train_accuracy_gnb = accuracy_score(y_train, y_train_pred_gnb) * 100
val_accuracy_gnb = accuracy_score(y_val, y_val_pred_gnb) * 100

print(f"MultinomialNB - Precisión en entrenamiento: {train_accuracy_gnb:.2f}%")
print(f"MultinomialNB - Precisión en validación: {val_accuracy_gnb:.2f}%")

MultinomialNB - Precisión en entrenamiento: 98.42%
MultinomialNB - Precisión en validación: 97.50%


In [26]:
from CustomNB import GaussianNaiveBayes
import numpy as np

model = GaussianNaiveBayes()
model.fit(np.array(X_train), np.array(y_train), method='MLE')

In [27]:
y_train_pred_cnb = model.predict(np.array(X_train))
y_val_pred_cnb = model.predict(np.array(X_val))

train_accuracy_cnb = accuracy_score(y_train, y_train_pred_cnb) * 100
val_accuracy_cnb = accuracy_score(y_val, y_val_pred_cnb) * 100

print(f"CustomNB - Precisión en entrenamiento: {train_accuracy_cnb:.2f}%")
print(f"CustomNB - Precisión en validación: {val_accuracy_cnb:.2f}%")

CustomNB - Precisión en entrenamiento: 83.83%
CustomNB - Precisión en validación: 83.25%
